In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [0]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
(features, labels), (X_test_orig, Y_test_orig) = cifar10.load_data()
features = features/255 - 0.5
labels = labels.flatten()

test_images = X_test_orig/255 - 0.5
test_labels = Y_test_orig.flatten()


In [0]:
#for stability
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

In [0]:
print('designing model')
# Training Parameters
learning_rate = 0.005
n_epochs = 2500 # changed to 2500 for a LB score of ~2.69


# Network Parameters
num_input = 32*32 #size of the images
num_classes = 10 # Binary
dropout = 0.4 # Dropout, probability to keep units

In [0]:
X = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")


with tf.variable_scope('ConvNet'):

    conv1 = tf.layers.conv2d(
      inputs=X,
      filters=32,
      kernel_size=[5, 5],
      padding="valid",
      activation=tf.nn.relu, name="conv1")

    conv2 = tf.layers.conv2d(conv1, filters=64, 
                         kernel_size=3,
                         strides=2, padding="valid",
                         activation=tf.nn.relu, name="conv2")
    pool3 = tf.nn.max_pool(conv2,
                       ksize=[1, 2, 2, 1],
                       strides=[1, 2, 2, 1],
                       padding="VALID")
    conv4 = tf.layers.conv2d(pool3, filters=128, 
                         kernel_size=4,
                         strides=3, padding="SAME",
                         activation=tf.nn.relu, name="conv4")

    pool5 = tf.nn.max_pool(conv4,
                       ksize=[1, 2, 2, 1],
                       strides=[1, 1, 1, 1],
                       padding="VALID")

    pool5_flat = tf.contrib.layers.flatten(pool5)

    fullyconn1 = tf.layers.dense(pool5_flat, 128,
                             activation=tf.nn.relu, name="fc1")

    fullyconn2 = tf.layers.dense(fullyconn1, 64,
                             activation=tf.nn.relu, name="fc2")
    logits = tf.layers.dense(fullyconn2, 10, name="output")

In [0]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [0]:
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()
#saver = tf.train.Saver()

In [0]:
print('training model\n')
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: features, y: labels})   
        acc_train = accuracy.eval(feed_dict={X: X_train, y: y_train})
        acc_test = accuracy.eval(feed_dict={X: X_valid,
                                            y: y_valid})
    
        print(epoch, "Train accuracy:", acc_train, "Validation accuracy:", acc_test)
    save_path = saver.save(sess, "./cam_iceberg_model_final.ckpt")

## Prepare the test data

As we did with the training and validation data, before making predictions I convert the type of the test data to float32.


In [0]:
#convert the test images to float32
test_images =test_images.astype(np.float32) 
test_images.shape


## Make predictions

The last line y_pred = Z[:,1] selects the second column of the predictions because we want 'probability of iceberg' not 'probability of not iceberg' which would be column 0.

In [0]:

print('making predictions\n')
#make external predictions on the test_dat
with tf.Session() as sess:
    saver.restore(sess, "./cam_iceberg_model_final.ckpt") # or better, use save_path
    Z = logits.eval(feed_dict={X: test_images}) #outputs switched to logits
    y_pred = Z[:,1]



## Write output to file

Lastly we take the predictions and construct a dataframe which we output to a .csv and can then submit for evalutation!

In [0]:
output = pd.DataFrame(test_df['id'])
output['is_iceberg'] = y_pred

output.to_csv('cam_tf_cnn.csv', index=False)